In [2]:
import os
import time
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import regex as re
import requests
import plotly.express as px
import plotly.graph_objects as go
import json
import datetime
from scipy import stats
import tqdm


In [3]:
#Change the city to the city you want to pull data for
city = 'forked-river-nj'

In [7]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

df1 = pd.DataFrame()
base_url = 'https://www.zillow.com/'+ city +'/sold/'  

for beds_count in [1, 2, 3, 4, 5]:
    for baths_count in [1.0, 1.5, 2.0, 3.0, 4.0]:
        #Loop through all 20 of zillows pages
        for i in tqdm.tqdm(range(1,20)):
            with requests.Session() as s:

                #sleep so that they think we're human
                time.sleep(np.random.randint(1,3))
                
                if i == 1:
                    page = ""
                else: 
                    page = str(i) + "_p/"

                bed_part =  str(beds_count) + "-_beds/"
                bath_part = str(baths_count) + "-_baths/"
                
                url = base_url + bed_part + bath_part + page
                
                patterns = [
                    r'<script id="__NEXT_DATA__" type="application/json">(.*?)</script>',
                    r'({"props":{"encodedZuid":"",.*?})',
                    r'<script type="application/json" data-zrr-shared-data-key="mobileSearchPageStore" ><!--(.*?)-->',
                ]

                r = s.get(url, headers=req_headers)

                match = None
                for pattern in patterns:
                    match = re.search(pattern, r.text, re.DOTALL)
                    if match:
                        break

                if match is None:
                    print(f"No match found on page {i} for beds: {beds_count}, baths: {baths_count}")
                else:
                    data_string = match.group(1)
                    try:
                        data = json.loads(data_string)
                    except json.JSONDecodeError as e:
                        print(f"Failed to parse JSON on page {i} for beds: {baths_count}, baths: {baths_count}: {e}")
                        print(f"Offending string: {data_string}")
                        continue

                    try:
                        df = pd.json_normalize(data['cat1']['searchResults']['listResults'])

                        # store in a dataframe
                        df1 = df1.append(df)
                    except Exception as e:
                        print(f"Failed to create DataFrame on page {i} for beds: {beds_count}, baths: {baths_count}: {e}")


  5%|▌         | 1/19 [00:02<00:50,  2.83s/it]

Failed to create DataFrame on page 1 for beds: 1, baths: 1.0: 'cat1'


 16%|█▌        | 3/19 [00:06<00:33,  2.09s/it]

Failed to create DataFrame on page 3 for beds: 1, baths: 1.0: 'cat1'


 26%|██▋       | 5/19 [00:10<00:28,  2.03s/it]

Failed to create DataFrame on page 5 for beds: 1, baths: 1.0: 'cat1'


 37%|███▋      | 7/19 [00:14<00:23,  2.00s/it]

Failed to create DataFrame on page 7 for beds: 1, baths: 1.0: 'cat1'


 42%|████▏     | 8/19 [00:17<00:23,  2.15s/it]


KeyboardInterrupt: 

In [13]:
pd.json_normalize(data['cat1']['searchResults']['listResults'])

,zpid,id,providerListingId,imgSrc,hasImage,carouselPhotos,detailUrl,statusType,statusText,countryCurrency,...,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.videoCount,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,streetViewMetadataURL,streetViewURL
0,39621144,39621144,None,https://photos.zillowstatic.com/fp/c952ada87d0...,True,[{'url': 'https://photos.zillowstatic.com/fp/c...,https://www.zillow.com/homedetails/403-Beach-B...,SOLD,Sold,$,...,132500.0,6098.400000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN
1,39621811,39621811,None,https://photos.zillowstatic.com/fp/6d61986d4e4...,True,[{'url': 'https://photos.zillowstatic.com/fp/6...,https://www.zillow.com/homedetails/1201-Beach-...,SOLD,Sold,$,...,229500.0,8002.000000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN
2,39623014,39623014,None,https://photos.zillowstatic.com/fp/f45c72f75a7...,True,[{'url': 'https://photos.zillowstatic.com/fp/f...,https://www.zillow.com/homedetails/230-Birch-L...,SOLD,Sold,$,...,242100.0,10001.000000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN
3,39624491,39624491,None,https://photos.zillowstatic.com/fp/bec84efa236...,True,[{'url': 'https://photos.zillowstatic.com/fp/b...,https://www.zillow.com/homedetails/1218-Ariel-...,SOLD,Sold,$,...,313400.0,NaN,NaN,False,1.0,NaN,NaN,NaN,NaN,NaN
4,39628397,39628397,None,https://photos.zillowstatic.com/fp/389a39edc02...,True,[{'url': 'https://photos.zillowstatic.com/fp/3...,https://www.zillow.com/homedetails/427-Penn-Av...,SOLD,Sold,$,...,186200.0,7840.800000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN
5,39628911,39628911,None,https://photos.zillowstatic.com/fp/6270070b174...,True,[{'url': 'https://photos.zillowstatic.com/fp/6...,https://www.zillow.com/homedetails/731-Alpine-...,SOLD,Sold,$,...,208400.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
6,39622715,39622715,None,https://photos.zillowstatic.com/fp/0666a3652ba...,True,[{'url': 'https://photos.zillowstatic.com/fp/0...,https://www.zillow.com/homedetails/14-Bridge-C...,SOLD,Sold,$,...,810300.0,1.160000,acres,False,NaN,NaN,NaN,NaN,NaN,NaN
7,39622235,39622235,None,https://photos.zillowstatic.com/fp/2e8b16298a6...,True,[{'url': 'https://photos.zillowstatic.com/fp/2...,https://www.zillow.com/homedetails/24-Hollywoo...,SOLD,Sold,$,...,356000.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
8,39623658,39623658,None,https://photos.zillowstatic.com/fp/1aaef933d72...,True,[{'url': 'https://photos.zillowstatic.com/fp/1...,https://www.zillow.com/homedetails/844-Spar-Dr...,SOLD,Sold,$,...,500600.0,7471.000000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN
9,39626928,39626928,None,https://photos.zillowstatic.com/fp/94d20456235...,True,[{'url': 'https://photos.zillowstatic.com/fp/9...,https://www.zillow.com/homedetails/1282-Spruce...,SOLD,Sold,$,...,222200.0,8712.000000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
re.search(r'<!--{.*?}', r.text).group(1)

IndexError: no such group

In [24]:
#pretty print r.text to see what the page looks like
re.search(r'<!--\{"queryState".*?-->', r.text)[0]

'<!--{"queryState":{"mapBounds":{"north":39.94004,"south":39.80442,"east":-74.139814,"west":-74.40421},"regionSelection":[{"regionId":38565,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForSaleForeclosure":{"value":false},"isRecentlySold":{"value":true},"sortSelection":{"value":"globalrelevanceex"}}},"filterDefinitions":{"keywords":{"id":"keywords","shortId":"att","labels":{"default":"Keywords","tracking":"Keyword"},"sortOrder":2,"type":"String","defaultValue":{"value":""},"exposedPillEnabled":true},"isPublicSchool":{"id":"isPublicSchool","shortId":"schp","labels":{"default":"Public"},"type":"Boolean","defaultValue":{"value":true}},"isCityView":{"id":"isCityView","shortId":"cityv","labels":{"default":"City"},"type":"Boolean","defaultValue":{"value":false}},"isWaterfront":{"id":"isWaterfront","shortId":"wat","labels

In [57]:
#Drop duplicates
df = df1.drop_duplicates(subset=['zpid'], keep='first')
df = df.reset_index(drop=True)  

In [ ]:
##Save CSV with today's date
today = time.strftime("%Y%m%d")
df.to_csv(city+'_'+today+'.csv')

In [2]:
#Pull in all past data for analysis
files = [f for f in os.listdir('.') if f.endswith('.csv')]

#Only keep files with city variable in them
files = [f for f in files if city in f]

#Combine all CSV files into one
df = pd.concat([pd.read_csv(f) for f in files])

#Remove duplicates
df = df.drop_duplicates(subset=['zpid'], keep='first')
df = df.reset_index(drop=True)

NameError: name 'city' is not defined

In [98]:
######Analyse price per sqft of single famil homes ######

#Only single family homes
df = df[df['hdpData.homeInfo.homeType'] == 'SINGLE_FAMILY']

#Subset data to drop rows with NAN in area
df = df[pd.notnull(df['area'])]

#sold_date to date
df['sold_date'] = pd.to_datetime(df['hdpData.homeInfo.dateSold'], unit='ms')

#Convert to int
df['price'] = df['unformattedPrice'].astype(int)
df['sqft'] = df['area'].astype(int)
df['beds'] = df['beds'].astype(int)
df['baths'] = df['baths'].astype(int)

#Remove Crazy outliers
df['sqft_zscore'] = stats.zscore(df['sqft'])
df = df[np.abs(df['sqft_zscore']) < 2]


#calculate the average price per sqft
df['price_sqft'] = df['price'] / df['sqft']

#create plotly scatter chart of avg price per sqft over time
fig = px.scatter(df, x='sold_date', y='price_sqft', color ='price',  
                color_continuous_scale=px.colors.sequential.Turbo, range_color=(100000, 1000000), title='Price Per Square Foot Over Time in '+city)
fig.update_traces(marker={'size': 5, 'opacity': 0.75})


#### Now group the data by week and create an EMA with an 8 week window

#Group data by day sold
df2 = df.groupby(['sold_date'], as_index=False)["price_sqft"].mean()

#add price ema
df2['price_ema'] = df2['price_sqft'].ewm(span=30).mean()

##Add trace to plotly fig
fig.add_traces(
    list(px.line(df2, x='sold_date', y='price_ema').select_traces())
    )
fig.update_traces(line_color='#303030', line_width=5)
fig.show()


In [90]:
###### Analyse the taxEstimate to sold price ratio  ######
df2 = df[pd.notnull(df['hdpData.homeInfo.taxAssessedValue'])]

#Create a new column with the taxEstimate to sold price ratio
df2['sptv_ratio'] = df2['price'] / df2['hdpData.homeInfo.taxAssessedValue']

#Remove major outliers
df2['sptv_zscore'] = stats.zscore(df2['sptv_ratio'])
df2 = df2[np.abs(df2['sptv_zscore']) < 1]

##### Plot Sold Price to Tax Estimate Ratio #####
fig = px.scatter(df2, x='sold_date', y='sptv_ratio', color ='price',  
                 color_continuous_scale=px.colors.sequential.Turbo, title='Sold Price to Tax Accessed Value over time in '+city)
fig.update_traces(marker={'size': 5, 'opacity': 0.75})

### Add moving EMA by week
##floor date by week
df3 = df2.groupby(['sold_date'], as_index=False)["sptv_ratio"].mean()

#add sptv_ratio ema
df3['sptv_ratio_ema'] = df3['sptv_ratio'].ewm(span=30).mean()

##Add EMA trace to plotly fig
fig.add_trace(go.Scatter(x=df3['sold_date'], y= df3['sptv_ratio_ema'], mode='lines', line_color='#303030', line_width=4))

fig.data = (fig.data[1],fig.data[0])
fig.show()

